In [5]:
import numpy as np
import pandas as pd  # Import pandas for saving to Excel
import time

# Parameters
N = 100  # Number of grid points
L = 1.0  # Length of the domain
phi_L = 0.0  # Boundary condition at x = 0
phi_R = 0.0  # Boundary condition at x = 1
dx = L / (N - 1)  # Grid spacing

# Create the grid points
x = np.linspace(0, L, N)

# Analytical solution (for comparison)
phi_analytical = (-x**4 / 12 + x**3 / 6 - x / 12)

# Define the source term s(x) = x * (1 - x)
s = x * (1 - x)

# Construct the right-hand side vector b
b = s.copy()
b[0] = phi_L
b[-1] = phi_R

# Method 1: Normal Direct Solver (Dense Matrix)
A_dense = np.zeros((N, N))
for i in range(1, N-1):
    A_dense[i, i-1] = 1
    A_dense[i, i] = -2
    A_dense[i, i+1] = 1
A_dense[0, 0] = 1
A_dense[-1, -1] = 1
phi_normal = np.linalg.solve(A_dense, b)/10000  # Solve the linear system

# Method 2: Sparse LU Decomposition
diagonals = [
    np.ones(N-1),        # Upper diagonal
    -2 * np.ones(N),     # Main diagonal
    np.ones(N-1)         # Lower diagonal
]
offsets = [-1, 0, 1]
A_sparse = diags(diagonals, offsets, format='csc')  # Create sparse matrix directly
A_sparse[0, 0] = 1.0
A_sparse[0, 1] = 0.0
A_sparse[-1, -1] = 1.0
A_sparse[-1, -2] = 0.0
lu = splu(A_sparse)  # LU decomposition for sparse matrix
phi_lu_sparse = lu.solve(b)/10000  # Solve the linear system

# Method 3: Gauss-Seidel with Sparse Matrix
phi_gs = np.zeros(N)
phi_gs[0] = phi_L
phi_gs[-1] = phi_R
max_iter = 5000
tolerance = 1e-4

# Gauss-Seidel iteration
for iteration in range(max_iter):
    phi_old = phi_gs.copy()
    for i in range(1, N-1):
        a_ii = A_sparse[i, i]
        a_ij_left = A_sparse[i, i-1]
        a_ij_right = A_sparse[i, i+1]
        phi_gs[i] = ((b[i] - a_ij_left * phi_gs[i-1] - a_ij_right * phi_gs[i+1]) / a_ii)/10000
    error = np.linalg.norm(phi_gs - phi_old, ord=np.inf)
    if error < tolerance:
        break

# Calculate absolute and relative errors for each method
error_abs_normal = np.abs(phi_normal - phi_analytical)
error_rel_normal = error_abs_normal / np.abs(phi_analytical)

error_abs_lu = np.abs(phi_lu_sparse - phi_analytical)
error_rel_lu = error_abs_lu / np.abs(phi_analytical)

error_abs_gs = np.abs(phi_gs - phi_analytical)
error_rel_gs = error_abs_gs / np.abs(phi_analytical)

# Prepare the data for Excel
data = {
    'x': x,
    'phi_analytical': phi_analytical,
    'phi_normal': phi_normal,
    'phi_lu_sparse': phi_lu_sparse,
    'phi_gs': phi_gs,
    'Error_abs_normal': error_abs_normal,
    'Error_rel_normal': error_rel_normal,
    'Error_abs_lu': error_abs_lu,
    'Error_rel_lu': error_rel_lu,
    'Error_abs_gs': error_abs_gs,
    'Error_rel_gs': error_rel_gs,
}

# Create a DataFrame to store the data
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df.to_excel('poisson_solution_with_errors.xlsx', index=False, engine='openpyxl')

print("Results with errors have been saved to 'poisson_solution_with_errors.xlsx'")



Results with errors have been saved to 'poisson_solution_with_errors.xlsx'


C:\Users\AmirHosein\AppData\Local\Temp\ipykernel_13184\1087110260.py:72: RuntimeWarning: invalid value encountered in divide
  error_rel_normal = error_abs_normal / np.abs(phi_analytical)
C:\Users\AmirHosein\AppData\Local\Temp\ipykernel_13184\1087110260.py:75: RuntimeWarning: invalid value encountered in divide
  error_rel_lu = error_abs_lu / np.abs(phi_analytical)
C:\Users\AmirHosein\AppData\Local\Temp\ipykernel_13184\1087110260.py:78: RuntimeWarning: invalid value encountered in divide
  error_rel_gs = error_abs_gs / np.abs(phi_analytical)
